In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from tensorflow.keras.models import load_model

2025-08-16 21:40:06.673033: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-16 21:40:06.681157: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-16 21:40:07.020841: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-16 21:40:09.098188: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
## loading the model.
model = load_model("model.h5")

## loading the scalar, label_encoder, geo_onehot encoder
with open('standard_scaler.pkl', 'rb') as file:
    scalar = pickle.load(file)
with open('label_encoder_gender.pkl','rb') as file:
    encoder_gender = pickle.load(file)
with open('geo_onehot_encoder.pkl', 'rb') as file:
    onehot_geo =pickle.load(file)

2025-08-16 21:40:10.710752: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}
df =pd.DataFrame([input_data])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [4]:
geo_encoded = onehot_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/home/aadharsh/Desktop/github/ANN_Classification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
df = pd.concat([df.drop(['Geography'],axis=1).reset_index(drop=True),geo_encoded_df],axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [6]:
## Encoding gender/ categorical variable
df['Gender'] = encoder_gender.transform(df['Gender'])
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [7]:
# Scaling the input data.
df = scalar.transform(df)
df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [8]:
prediction = model.predict(df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


array([[0.04209425]], dtype=float32)

In [9]:
prediction_value = prediction[0][0]
prediction_value

np.float32(0.042094246)

In [10]:
if prediction_value>0.5:
    print("The Customer is likely to churn")
else:
    print("The Customer is not likely to churn")

The Customer is not likely to churn
